In [ ]:
import marimo as mo
import polars as pl

# PPA excerpt changes


A spreadsheet from 2023-09 was used to adjust the page ranges for excerpts that had shifted due to updates in HathiTrust content.  That manual data work provides a window into the kinds of changes seen in PPA content.

In [ ]:
excerpt_df = pl.read_csv("data/ppa/excerpts-2023-09-20.csv")
excerpt_df

<marimo-table data-initial-value='[]' data-label='null' data-data='"[{\"source_url\":\"https://hdl.handle.net/2027/hvd.32044090278565\",\"title\":\"Abbot \\u00c6lfric's rhythmic prose.\",\"author\":\"Gerould, Gordon Hall, 1877-\",\"book_journal\":\"MP\",\"pub_date\":1925,\"pub_place\":\"[Chicago\",\"publisher\":\"University of Chicago Press\",\"enumcron\":null,\"pages_orig\":\"353-366\",\"pages_digital\":\"5-18\",\"new digital range\":\"correct\",\"page_count\":14,\"added\":\"2022-01-25 15:02:04\",\"updated\":\"2022-01-25 15:04:16\",\"item_type\":\"Article\",\"source_id\":\"hvd.32044090278565\",\"\":null,\"side notes (poetry, couldn't find article, item unavailable, original pages differentiate)\":null},{\"source_url\":\"https://hdl.handle.net/2027/nyp.33433081683744\",\"title\":\"About Rhyme\",\"author\":\"Benton, Joel\",\"book_journal\":\"Lippincott's monthly magazine.\",\"pub_date\":1889,\"pub_place\":\"Philadelphia, Pa.\",\"publisher\":\"J.B. Lippincott Co.\",\"enumcron\":\"v. 43 (Jan.-June 1889)\",\"pages_orig\":\"901-4\",\"pages_digital\":\"929-932\",\"new digital range\":\"955-958\",\"page_count\":4,\"added\":\"2018-01-23 15:04:23\",\"updated\":\"2021-08-05 09:52:05\",\"item_type\":\"Article\",\"source_id\":\"nyp.33433081683744\",\"\":null,\"side notes (poetry, couldn't find article, item unavailable, original pages differentiate)\":null},{\"source_url\":\"https://hdl.handle.net/2027/uc1.b3924132\",\"title\":\"Accent a Guiding Principle, not Merely of the Old Comic Metres, but Generally of Latin Poetry; and First of Virgil's Hexameters\",\"author\":\"Key, T. Hewitt\",\"book_journal\":\"Transactions of the Philological Society\",\"pub_date\":1874,\"pub_place\":\"Oxford [etc.]\",\"publisher\":\"Society by B. Blackwell\",\"enumcron\":\"1873/74\",\"pages_orig\":\"35-52\",\"pages_digital\":\"45-62\",\"new digital range\":\"correct\",\"page_count\":18,\"added\":\"2018-01-23 15:09:51\",\"updated\":\"2021-08-05 09:52:08\",\"item_type\":\"Article\",\"source_id\":\"uc1.b3924132\",\"\":null,\"side notes (poetry, couldn't find article, item unavailable, original pages differentiate)\":null},{\"source_url\":\"https://hdl.handle.net/2027/mdp.39015026482151\",\"title\":\"Accent and Ictus in the Latin Elegiac Distich\",\"author\":\"Sturtevant, E. H.\",\"book_journal\":\"Transactions and proceedings of the American Philological Association\",\"pub_date\":1924,\"pub_place\":\"Cleveland, etc.\",\"publisher\":\"Association by the Press of Case Western Reserve University [etc.]\",\"enumcron\":\"v.55\",\"pages_orig\":\"73-89\",\"pages_digital\":\"81-97\",\"new digital range\":\"correct\",\"page_count\":17,\"added\":\"2021-08-05 09:52:15\",\"updated\":\"2021-08-05 09:52:15\",\"item_type\":\"Article\",\"source_id\":\"mdp.39015026482151\",\"\":null,\"side notes (poetry, couldn't find article, item unavailable, original pages differentiate)\":null},{\"source_url\":\"https://hdl.handle.net/2027/uiug.30112106245936\",\"title\":\"Accent Collation of Caedmon's Genesis B\",\"author\":\"Stoddard, Francis H.\",\"book_journal\":\"Modern language notes.\",\"pub_date\":1886,\"pub_place\":\"Baltimore\",\"publisher\":\"Johns Hopkins Press [etc.]\",\"enumcron\":\"v.2(1886)\",\"pages_orig\":\"165-73\",\"pages_digital\":\"93-97\",\"new digital range\":\"correct\",\"page_count\":5,\"added\":\"2018-01-23 15:15:13\",\"updated\":\"2021-08-05 09:52:12\",\"item_type\":\"Article\",\"source_id\":\"uiug.30112106245936\",\"\":null,\"side notes (poetry, couldn't find article, item unavailable, original pages differentiate)\":null},{\"source_url\":\"https://hdl.handle.net/2027/hvd.32044009576562\",\"title\":\"Accentual Rhythm in Latin\",\"author\":\"Greenough, J. B\",\"book_journal\":\"Harvard studies in classical philology\",\"pub_date\":1893,\"pub_place\":\"Cambridge, Mass. [etc.]\",\"publisher\":\"Harvard University Press [etc.]\",\"enumcron\":\"v.4 (1893)\",\"pages_orig\":\"105-15\",\"pages_digital\":\"117-127\",\"new digital range\":\"correct\",\"page_count\":11,\"ad

In [ ]:
changed = excerpt_df.filter(pl.col("new digital range").ne("correct"))
total_excerpts = excerpt_df.height
total_changed = changed.height
total_unchanged = total_excerpts - total_changed
percent_unchanged = (total_unchanged / total_excerpts) * 100
percent_changed = (total_changed / total_excerpts) * 100

mo.md(f"""
Of {total_excerpts:,} total excerpts, there were {total_unchanged:,} excerpts ({percent_unchanged:.1f}%) where the digital page range was unchanged.

{total_changed:,} excerpts had changed ({percent_changed:.1f}%).
""")

Of 517 total excerpts, there were 396 excerpts (76.6%) where the digital page range was unchanged. 
 121 excerpts had changed (23.4%).

For the excerpts that changed, how much did they change?

In [ ]:
from intspan import intspan

# limit to page ranges and page count
# parse new and old digital page range with int span so we can compare
changed_pages = (
    changed.filter(pl.col("new digital range").ne("SUPPRESS"))
    .select(
        "source_url",
        "title",
        "author",
        "pages_orig",
        "pages_digital",
        "new digital range",
        "page_count",
    )
    .with_columns(
        pl.col("pages_digital")
        .map_elements(lambda x: intspan(x), return_dtype=object)
        .alias("pages_digital_orig"),
        pl.col("new digital range")
        .map_elements(lambda x: intspan(x), return_dtype=object)
        .alias("pages_digital_new"),
    )
)

# calculate page count for new/old and compare
changed_pages = (
    changed_pages.with_columns(
        # convert intspans into list of pages, to simplify further calculations,
        # since polars can work easily with lists
        orig_pages=pl.col("pages_digital_orig").map_elements(
            lambda x: list(x),
            return_dtype=pl.datatypes.List(pl.datatypes.Int32),
        ),
        new_pages=pl.col("pages_digital_new").map_elements(
            lambda x: list(x),
            return_dtype=pl.datatypes.List(pl.datatypes.Int32),
        ),
    )
    .with_columns(
        orig_page_count=pl.col("orig_pages").list.len(),
        new_page_count=pl.col("new_pages").list.len(),
        orig_start_page=pl.col("orig_pages").list.first(),
        new_start_page=pl.col("new_pages").list.first(),
    )
    .with_columns(
        # cast to same integer type to avoid getting a weird signed/unsigned int math bogus result
        page_count_diff=pl.col("orig_page_count")
        .cast(pl.Int32)
        .sub(pl.col("new_page_count").cast(pl.Int32))
        .abs(),
        start_page_diff=pl.col("orig_start_page")
        .sub(pl.col("new_start_page"))
        .abs(),
        # use set intersection to determine # overlapping pages between old and new page range
        common_pages=pl.col("orig_pages").list.set_intersection(
            pl.col("new_pages")
        ),
    )
    # then count the pages included in both
    .with_columns(
        num_common_pages=pl.col("common_pages").list.len(),
    )
    .with_columns(
        # also calculate as a percentage of the excerpt, since they vary in size; use the new/corrected page count
        pct_common_pages=pl.col("num_common_pages").truediv(
            pl.col("new_page_count")
        )
    )
)

changed_pages

[E 250716 11:41:26 functions:66](B Error calling function get_column_summaries: cannot write 'Object' datatype to ipc


For a few excerpts, the number of pages included in the excerpt changed.

There are a couple of outliers with very large changes (25 and 16 page differences); omit them for reporting, since they are likely due to data errors in the original excerpt.

In [ ]:
page_count_changed = changed_pages.filter(
    pl.col("page_count_diff").ne(0)
).filter(pl.col("page_count_diff").lt(15))
page_count_changed["page_count_diff"].describe()

<marimo-table data-initial-value='[]' data-label='null' data-data='"[{\"statistic\":\"count\",\"value\":8.0},{\"statistic\":\"null_count\",\"value\":0.0},{\"statistic\":\"mean\",\"value\":1.75},{\"statistic\":\"std\",\"value\":0.4629100498862757},{\"statistic\":\"min\",\"value\":1.0},{\"statistic\":\"25%\",\"value\":2.0},{\"statistic\":\"50%\",\"value\":2.0},{\"statistic\":\"75%\",\"value\":2.0},{\"statistic\":\"max\",\"value\":2.0}]"' data-total-rows='9' data-total-columns='2' data-max-columns='50' data-banner-text='""' data-pagination='true' data-page-size='10' data-field-types='[["statistic", ["string", "str"]], ["value", ["number", "f64"]]]' data-show-filters='true' data-show-download='true' data-show-column-summaries='false' data-row-headers='[]' data-has-stable-row-id='false' data-lazy='false' data-preload='false'>

In [ ]:
mo.md(
    ", ".join(
        [str(n) for n in page_count_changed["page_count_diff"].to_list()]
    )
)

2, 2, 2, 2, 2, 1, 1, 2

In [ ]:
mo.md(
    f"""
For {page_count_changed.height} excerpts, the number of pages changed.

- The largest change was {page_count_changed["page_count_diff"].max()} pages.
- The average change was {page_count_changed["page_count_diff"].mean()} pages.
- The most common change was {page_count_changed["page_count_diff"].mode().first()} pages.
"""
)

For 8 excerpts, the number of pages changed. 
 
 The largest change was 2 pages. 
 The average change was 1.75 pages. 
 The most common change was 2 pages.

When excerpts shifted, how many pages did they typically shift?

In [ ]:
changed_pages["start_page_diff"].describe()

<marimo-table data-initial-value='[]' data-label='null' data-data='"[{\"statistic\":\"count\",\"value\":120.0},{\"statistic\":\"null_count\",\"value\":0.0},{\"statistic\":\"mean\",\"value\":6.391666666666667},{\"statistic\":\"std\",\"value\":22.15730462983672},{\"statistic\":\"min\",\"value\":0.0},{\"statistic\":\"25%\",\"value\":2.0},{\"statistic\":\"50%\",\"value\":2.0},{\"statistic\":\"75%\",\"value\":4.0},{\"statistic\":\"max\",\"value\":240.0}]"' data-total-rows='9' data-total-columns='2' data-max-columns='50' data-banner-text='""' data-pagination='true' data-page-size='10' data-field-types='[["statistic", ["string", "str"]], ["value", ["number", "f64"]]]' data-show-filters='true' data-show-download='true' data-show-column-summaries='false' data-row-headers='[]' data-has-stable-row-id='false' data-lazy='false' data-preload='false'>

In [ ]:
import altair as alt

alt.Chart(changed_pages).mark_bar(width=10).encode(
    x=alt.X("start_page_diff", title="Difference in start pages"), y="count()"
)

Maybe a box plot will be an easier way to show the distribution of changes.

In [ ]:
alt.Chart(changed_pages).mark_boxplot().encode(
    alt.X("start_page_diff", title="Difference in start pages")
)

In [ ]:
alt.Chart(
    changed_pages.filter(pl.col("start_page_diff").lt(100))
).mark_boxplot().encode(
    alt.X(
        "start_page_diff",
        title="Difference in start pages (240 page difference excluded)",
    )
)

In [ ]:
mo.md(
    f"""
Differences in start page:

- The largest change was {changed_pages["start_page_diff"].max()} pages.
- The average change was {changed_pages["start_page_diff"].mean():.1f} pages.
- The most common change was {changed_pages["start_page_diff"].mode().first()} pages.
"""
)

Differences in start page: 
 
 The largest change was 240 pages. 
 The average change was 6.4 pages. 
 The most common change was 2 pages.

In [ ]:
changed_pages.filter(pl.col("start_page_diff").gt(10))

<marimo-table data-initial-value='[]' data-label='null' data-data='"[{\"source_url\":\"https://hdl.handle.net/2027/nyp.33433081683744\",\"title\":\"About Rhyme\",\"author\":\"Benton, Joel\",\"pages_orig\":\"901-4\",\"pages_digital\":\"929-932\",\"new digital range\":\"955-958\",\"page_count\":4,\"pages_digital_orig\":\"929-932\",\"pages_digital_new\":\"955-958\",\"orig_pages\":[929,930,931,932],\"new_pages\":[955,956,957,958],\"orig_page_count\":4,\"new_page_count\":4,\"orig_start_page\":929,\"new_start_page\":955,\"page_count_diff\":0,\"start_page_diff\":26,\"common_pages\":[],\"num_common_pages\":0,\"pct_common_pages\":0.0},{\"source_url\":\"https://hdl.handle.net/2027/mdp.39015060425942\",\"title\":\"Contributions to the Study of Homeric Metre.\",\"author\":\"Bolling, G. M\",\"pages_orig\":\"401-25\",\"pages_digital\":\"497-519\",\"new digital range\":\"521-545\",\"page_count\":23,\"pages_digital_orig\":\"497-519\",\"pages_digital_new\":\"521-545\",\"orig_pages\":[497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519],\"new_pages\":[521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545],\"orig_page_count\":23,\"new_page_count\":25,\"orig_start_page\":497,\"new_start_page\":521,\"page_count_diff\":2,\"start_page_diff\":24,\"common_pages\":[],\"num_common_pages\":0,\"pct_common_pages\":0.0},{\"source_url\":\"https://hdl.handle.net/2027/uc1.b2905408\",\"title\":\"Coventry Patmore's Literary Criticism\",\"author\":\"Baum, P. F. \",\"pages_orig\":\"244-60\",\"pages_digital\":\"272-288\",\"new digital range\":\"292-308\",\"page_count\":17,\"pages_digital_orig\":\"272-288\",\"pages_digital_new\":\"292-308\",\"orig_pages\":[272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288],\"new_pages\":[292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308],\"orig_page_count\":17,\"new_page_count\":17,\"orig_start_page\":272,\"new_start_page\":292,\"page_count_diff\":0,\"start_page_diff\":20,\"common_pages\":[],\"num_common_pages\":0,\"pct_common_pages\":0.0},{\"source_url\":\"https://hdl.handle.net/2027/mdp.39015033845549\",\"title\":\"English Hexameter Exemplified\",\"author\":\"Taylor, William.\",\"pages_orig\":\"404-405\",\"pages_digital\":\"404-405\",\"new digital range\":\"428-429\",\"page_count\":2,\"pages_digital_orig\":\"404-405\",\"pages_digital_new\":\"428-429\",\"orig_pages\":[404,405],\"new_pages\":[428,429],\"orig_page_count\":2,\"new_page_count\":2,\"orig_start_page\":404,\"new_start_page\":428,\"page_count_diff\":0,\"start_page_diff\":24,\"common_pages\":[],\"num_common_pages\":0,\"pct_common_pages\":0.0},{\"source_url\":\"https://hdl.handle.net/2027/mdp.39015043572422\",\"title\":\"The 'Five Types' in Anglo-Saxon Verse\",\"author\":\"Greg, W. W.\",\"pages_orig\":\"12-17\",\"pages_digital\":\"12-33\",\"new digital range\":\"28-33\",\"page_count\":22,\"pages_digital_orig\":\"12-33\",\"pages_digital_new\":\"28-33\",\"orig_pages\":[12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33],\"new_pages\":[28,29,30,31,32,33],\"orig_page_count\":22,\"new_page_count\":6,\"orig_start_page\":12,\"new_start_page\":28,\"page_count_diff\":16,\"start_page_diff\":16,\"common_pages\":[28,29,30,31,32,33],\"num_common_pages\":6,\"pct_common_pages\":1.0},{\"source_url\":\"https://hdl.handle.net/2027/njp.32101076472966\",\"title\":\"The Invention of the Sonnet\",\"author\":\"Wilkins, E. H\",\"pages_orig\":\"463-94\",\"pages_digital\":\"493-524\",\"new digital range\":\"475-506\",\"page_count\":32,\"pages_digital_orig\":\"493-524\",\"pages_digital_new\":\"475-506\",\"orig_pages\":[493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524],\"new_pages\":[475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506],\"orig_page_count\":32,\"new_page_count\":32,\"orig_start_page\":493,\"new_start_

Nine volumes have a difference of more than 10 pages.

In [ ]:
changed_pages.filter(pl.col("start_page_diff").gt(20))

<marimo-table data-initial-value='[]' data-label='null' data-data='"[{\"source_url\":\"https://hdl.handle.net/2027/nyp.33433081683744\",\"title\":\"About Rhyme\",\"author\":\"Benton, Joel\",\"pages_orig\":\"901-4\",\"pages_digital\":\"929-932\",\"new digital range\":\"955-958\",\"page_count\":4,\"pages_digital_orig\":\"929-932\",\"pages_digital_new\":\"955-958\",\"orig_pages\":[929,930,931,932],\"new_pages\":[955,956,957,958],\"orig_page_count\":4,\"new_page_count\":4,\"orig_start_page\":929,\"new_start_page\":955,\"page_count_diff\":0,\"start_page_diff\":26,\"common_pages\":[],\"num_common_pages\":0,\"pct_common_pages\":0.0},{\"source_url\":\"https://hdl.handle.net/2027/mdp.39015060425942\",\"title\":\"Contributions to the Study of Homeric Metre.\",\"author\":\"Bolling, G. M\",\"pages_orig\":\"401-25\",\"pages_digital\":\"497-519\",\"new digital range\":\"521-545\",\"page_count\":23,\"pages_digital_orig\":\"497-519\",\"pages_digital_new\":\"521-545\",\"orig_pages\":[497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519],\"new_pages\":[521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545],\"orig_page_count\":23,\"new_page_count\":25,\"orig_start_page\":497,\"new_start_page\":521,\"page_count_diff\":2,\"start_page_diff\":24,\"common_pages\":[],\"num_common_pages\":0,\"pct_common_pages\":0.0},{\"source_url\":\"https://hdl.handle.net/2027/mdp.39015033845549\",\"title\":\"English Hexameter Exemplified\",\"author\":\"Taylor, William.\",\"pages_orig\":\"404-405\",\"pages_digital\":\"404-405\",\"new digital range\":\"428-429\",\"page_count\":2,\"pages_digital_orig\":\"404-405\",\"pages_digital_new\":\"428-429\",\"orig_pages\":[404,405],\"new_pages\":[428,429],\"orig_page_count\":2,\"new_page_count\":2,\"orig_start_page\":404,\"new_start_page\":428,\"page_count_diff\":0,\"start_page_diff\":24,\"common_pages\":[],\"num_common_pages\":0,\"pct_common_pages\":0.0},{\"source_url\":\"https://hdl.handle.net/2027/uc1.c2641998\",\"title\":\"Swinburne as a Metrician\",\"author\":\"Omond, Thomas Stewart\",\"pages_orig\":\"32-33\",\"pages_digital\":\"162-163, 196\",\"new digital range\":\"402-403\",\"page_count\":3,\"pages_digital_orig\":\"162-163,196\",\"pages_digital_new\":\"402-403\",\"orig_pages\":[162,163,196],\"new_pages\":[402,403],\"orig_page_count\":3,\"new_page_count\":2,\"orig_start_page\":162,\"new_start_page\":402,\"page_count_diff\":1,\"start_page_diff\":240,\"common_pages\":[],\"num_common_pages\":0,\"pct_common_pages\":0.0},{\"source_url\":\"https://hdl.handle.net/2027/mdp.39015087701341\",\"title\":\"Verse\",\"author\":\"Gosse, Edmund\",\"pages_orig\":\"1041-47\",\"pages_digital\":\"1091-1097\",\"new digital range\":\"1125-1133\",\"page_count\":7,\"pages_digital_orig\":\"1091-1097\",\"pages_digital_new\":\"1125-1133\",\"orig_pages\":[1091,1092,1093,1094,1095,1096,1097],\"new_pages\":[1125,1126,1127,1128,1129,1130,1131,1132,1133],\"orig_page_count\":7,\"new_page_count\":9,\"orig_start_page\":1091,\"new_start_page\":1125,\"page_count_diff\":2,\"start_page_diff\":34,\"common_pages\":[],\"num_common_pages\":0,\"pct_common_pages\":0.0}]"' data-total-rows='5' data-total-columns='20' data-max-columns='50' data-banner-text='""' data-pagination='true' data-page-size='10' data-field-types='[["source_url", ["string", "str"]], ["title", ["string", "str"]], ["author", ["string", "str"]], ["pages_orig", ["string", "str"]], ["pages_digital", ["string", "str"]], ["new digital range", ["string", "str"]], ["page_count", ["integer", "i64"]], ["pages_digital_orig", ["unknown", "object"]], ["pages_digital_new", ["unknown", "object"]], ["orig_pages", ["unknown", "list[i32]"]], ["new_pages", ["unknown", "list[i32]"]], ["orig_page_count", ["integer", "u32"]], ["new_page_count", ["integer", "u32"]], ["orig_start_page", ["integer", "i32"]], ["new_start_page", ["integer", "i32"]], ["page_count_diff", ["integer", "i32"]], ["start_page_diff", ["integer", "i32"]], ["common_pa

Five volumes have a difference of more than 20 pages; one has a difference of 240.

How many pages do the old and new ranges typically have in common?

In [ ]:
changed_pages["num_common_pages"].describe()

<marimo-table data-initial-value='[]' data-label='null' data-data='"[{\"statistic\":\"count\",\"value\":120.0},{\"statistic\":\"null_count\",\"value\":0.0},{\"statistic\":\"mean\",\"value\":18.458333333333332},{\"statistic\":\"std\",\"value\":36.171137533621895},{\"statistic\":\"min\",\"value\":0.0},{\"statistic\":\"25%\",\"value\":0.0},{\"statistic\":\"50%\",\"value\":8.0},{\"statistic\":\"75%\",\"value\":19.0},{\"statistic\":\"max\",\"value\":247.0}]"' data-total-rows='9' data-total-columns='2' data-max-columns='50' data-banner-text='""' data-pagination='true' data-page-size='10' data-field-types='[["statistic", ["string", "str"]], ["value", ["number", "f64"]]]' data-show-filters='true' data-show-download='true' data-show-column-summaries='false' data-row-headers='[]' data-has-stable-row-id='false' data-lazy='false' data-preload='false'>

In [ ]:
alt.Chart(changed_pages).mark_bar(width=10).encode(
    x=alt.X(
        "num_common_pages",
        title="Number of pages in common between new and old digital page range",
    ),
    y="count()",
)

In [ ]:
alt.Chart(changed_pages).mark_boxplot().encode(
    x=alt.X(
        "num_common_pages",
        title="# pages in common",
    )
).properties(
    title="Number of pages in common between new and old digital page range"
)

In [ ]:
alt.Chart(changed_pages).mark_bar(width=10).encode(
    x=alt.X(
        "pct_common_pages",
        title="Percent of pages in common between new and old digital page range",
    ),
    y="count()",
)

In [ ]:
no_pages_in_common = changed_pages.filter(
    pl.col("num_common_pages").eq(0)
).height


mo.md(
    f"{no_pages_in_common} excerpts have NO pages in common with the updated digital page range."
)

36 excerpts have NO pages in common with the updated digital page range.